# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [9]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [10]:
# TODO: Import the necessary libs
# For example: 
import os
from dotenv import load_dotenv
from lib.agents import Agent
from lib.llm import LLM
from lib.messages import UserMessage, SystemMessage, ToolMessage, AIMessage
from lib.tooling import Tool, ToolCall, tool
from lib.parsers import PydanticOutputParser, JsonOutputParser
from pydantic import BaseModel, Field

In [11]:
# TODO: Load environment variables
load_dotenv('/home/level-3/udacity/AgenticAI/.env/config.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
CHROMA_OPENAI_API_KEY = os.getenv("CHROMA_OPENAI_API_KEY")

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [12]:
# TODO: Create retrieve_game tool
# It should use chroma client and collection you created
import chromadb


chroma_client = chromadb.PersistentClient(path="/home/level-3/udacity/AgenticAI-Project-3-UdaPlay-An_AI_Research_Agent_for_the_Video_Game_Industry/starter/chromadb")
collection = chroma_client.get_collection("udaplay")

@tool
def retrieve_game(query: str) -> list:
    results = collection.query(
        query_texts=[query],
        n_results=3,
        )
    return results["metadatas"][0]  # Return the metadata of the results
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - query: a question about game industry. 
#
#    You'll receive results as list. Each element contains:
#    - Platform: like Game Boy, Playstation 5, Xbox 360...)
#    - Name: Name of the Game
#    - YearOfRelease: Year when that game was released for that platform
#    - Description: Additional details about the game
evaluation_question = "Best selling game of all time"
games_list = retrieve_game(evaluation_question)  # Example call to test the function
len(games_list)  # Should return 5 
games_list  # Should return a dictionary with keys: Platform, Name, YearOfRelease, Description

[{'Name': 'Super Mario 64',
  'Platform': 'Nintendo 64',
  'Publisher': 'Nintendo',
  'YearOfRelease': 1996,
  'Genre': 'Platformer',
  'Description': "A groundbreaking 3D platformer that set new standards for the genre, featuring Mario's quest to rescue Princess Peach."},
 {'Description': 'A comprehensive racing simulator featuring a vast selection of vehicles and tracks, with realistic driving physics.',
  'Name': 'Gran Turismo 5',
  'Genre': 'Racing',
  'Platform': 'PlayStation 3',
  'YearOfRelease': 2010,
  'Publisher': 'Sony Computer Entertainment'},
 {'Description': 'A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.',
  'Name': 'Gran Turismo',
  'YearOfRelease': 1997,
  'Publisher': 'Sony Computer Entertainment',
  'Genre': 'Racing',
  'Platform': 'PlayStation 1'}]

#### Evaluate Retrieval Tool

In [13]:
# TODO: Create evaluate_retrieval tool
# You might use an LLM as judge in this tool to evaluate the performance
# You need to prompt that LLM with something like:
# "Your task is to evaluate if the documents are enough to respond the query. "
# "Give a detailed explanation, so it's possible to take an action to accept it or not."
# Use EvaluationReport to parse the result
# Tool Docstring:
#    Based on the user's question and on the list of retrieved documents, 
#    it will analyze the usability of the documents to respond to that question. 
#    args: 
#    - question: original question from user
#    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
#    The result includes:
#    - useful: whether the documents are useful to answer the question
#    - description: description about the evaluation result

class EvaluationReport(BaseModel):
    """Pydantic model for video game data evaluation report."""
    
    useful: bool = Field(..., description="Whether the documents are useful to answer the question")
    description: str = Field(..., description="Description about the evaluation result")
    

    


@tool
def evaluate_retrieval(question: str, retrieved_docs: list) -> EvaluationReport:
    
    llm = LLM(model="gpt-4o-mini", api_key=OPENAI_API_KEY)
    
    role = "evaluate if the documents are enough to respond the query"
    
    instructions = F"""Give a detailed explanation, so it's possible to take an action to accept it or not.
    Question: {question}
    Retrieved documents: {retrieved_docs}
    Provide your answer in the following JSON format:
    {{
        "useful": <true/false>,
        "description": "<detailed explanation>"
    }}"""
    
    messages = [
            SystemMessage(
                content=(
                    f"You're an AI Agent and your role is {role}. "  
                    f"Your instructions: {instructions}"
                )
            )
        ]

        
    # Get AI response with structured format

    ai_message = llm.invoke(input=messages, response_format=EvaluationReport)
    parser = JsonOutputParser()
    return parser.parse(ai_message)

    
    


evaluate_retrieval(evaluation_question, games_list)  # Example call to test the function

{'useful': False,
 'description': "The retrieved documents provide information about three specific video games: 'Super Mario 64', 'Gran Turismo 5', and 'Gran Turismo'. However, none of these documents explicitly state which game is the best-selling game of all time. The question requires a definitive answer regarding the best-selling game, which typically involves sales figures or rankings that are not present in the provided documents. Therefore, the documents are not sufficient to respond to the query."}

#### Game Web Search Tool

In [14]:
# TODO: Create game_web_search tool
# Please use Tavily client to search the web
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - question: a question about game industry. 

from datetime import datetime
from typing import Dict
from tavily import TavilyClient


@tool
def web_search(query: str, search_depth: str = "advanced") -> Dict:
    """
    Search the web using Tavily API
    args:
        query (str): Search query
        search_depth (str): Type of search - 'basic' or 'advanced' (default: advanced)
    """
    api_key = os.getenv("TAVILY_API_KEY")
    client = TavilyClient(api_key=api_key)

    # Perform the search
    search_result = client.search(
        query=query,
        search_depth=search_depth,
        include_answer=True,
        include_raw_content=False,
        include_images=False,
    )

    # Format the results
    formatted_results = {
        "answer": search_result.get("answer", ""),
        "results": search_result.get("results", []),
        "search_metadata": {"timestamp": datetime.now().isoformat(), "query": query},
    }

    return formatted_results

### Agent

In [22]:
# TODO: Create your Agent abstraction using StateMachine
# Equip with an appropriate model
# Craft a good set of instructions
# Plug all Tools you developed

import json
from typing import List, Optional, Union
from lib.agents import AgentState
from lib.state_machine import StateMachine, Step, EntryPoint, Termination, Run
from lib.llm import LLM
from lib.messages import AIMessage, UserMessage, SystemMessage, ToolMessage, BaseMessage
from lib.tooling import Tool, ToolCall, tool
from lib.memory import ShortTermMemory


class MemoryAgent:
    def __init__(
        self,
        model_name: str,
        instructions: str,
        tools: List[Tool] = None,
        temperature: float = 0.7,
    ):
        """
        Initialize a MemoryAgent instance

        Args:
            model_name: Name/identifier of the LLM model to use
            instructions: System instructions for the agent
            tools: Optional list of tools available to the agent
            temperature: Temperature parameter for LLM (default: 0.7)
        """
        self.instructions = instructions
        self.tools = tools if tools else []
        self.model_name = model_name
        self.temperature = temperature

        # Initialize memory and state machine
        self.memory = ShortTermMemory()
        self.workflow = self._create_state_machine()

    def _prepare_messages_step(self, state: AgentState) -> AgentState:
        """Step logic: Prepare messages for LLM consumption"""
        messages = state.get("messages", [])

        # If no messages exist, start with system message
        if not messages:
            messages = [SystemMessage(content=state["instructions"])]

        # Add the new user message
        messages.append(UserMessage(content=state["user_query"]))

        return {"messages": messages, "session_id": state["session_id"]}

    def _llm_step(self, state: AgentState) -> AgentState:
        """Step logic: Process the current state through the LLM"""
        # Initialize LLM
        llm = LLM(model=self.model_name, temperature=self.temperature, tools=self.tools)

        response = llm.invoke(state["messages"])
        tool_calls = response.tool_calls if response.tool_calls else None

        # Create AI message with content and tool calls
        ai_message = AIMessage(content=response.content, tool_calls=tool_calls)

        return {
            "messages": state["messages"] + [ai_message],
            "current_tool_calls": tool_calls,
            "session_id": state["session_id"],
        }

    def _tool_step(self, state: AgentState) -> AgentState:
        """Step logic: Execute any pending tool calls"""
        tool_calls = state["current_tool_calls"] or []
        tool_messages = []

        for call in tool_calls:
            # Access tool call data correctly
            function_name = call.function.name
            function_args = json.loads(call.function.arguments)
            tool_call_id = call.id
            # Find the matching tool
            tool = next((t for t in self.tools if t.name == function_name), None)
            if tool:
                result = tool(**function_args)
                tool_message = ToolMessage(
                    content=json.dumps(result),
                    tool_call_id=tool_call_id,
                    name=function_name,
                )
                tool_messages.append(tool_message)

        # Clear tool calls and add results to messages
        return {
            "messages": state["messages"] + tool_messages,
            "current_tool_calls": None,
            "session_id": state["session_id"],
        }

    def _create_state_machine(self) -> StateMachine[AgentState]:
        """Create the internal state machine for the agent"""
        machine = StateMachine[AgentState](AgentState)

        # Create steps
        entry = EntryPoint[AgentState]()
        message_prep = Step[AgentState]("message_prep", self._prepare_messages_step)
        llm_processor = Step[AgentState]("llm_processor", self._llm_step)
        tool_executor = Step[AgentState]("tool_executor", self._tool_step)
        termination = Termination[AgentState]()

        machine.add_steps(
            [entry, message_prep, llm_processor, tool_executor, termination]
        )

        # Add transitions
        machine.connect(entry, message_prep)
        machine.connect(message_prep, llm_processor)

        # Transition based on whether there are tool calls
        def check_tool_calls(state: AgentState) -> Union[Step[AgentState], str]:
            """Transition logic: Check if there are tool calls"""
            if state.get("current_tool_calls"):
                return tool_executor
            return termination

        machine.connect(llm_processor, [tool_executor, termination], check_tool_calls)
        machine.connect(
            tool_executor, llm_processor
        )  # Go back to llm after tool execution

        return machine

    def invoke(self, query: str, session_id: Optional[str] = None) -> Run:
        """
        Run the agent on a query

        Args:
            query: The user's query to process
            session_id: Optional session identifier (uses "default" if None)

        Returns:
            The final run object after processing
        """
        session_id = session_id or "default"

        # Create session if it doesn't exist
        self.memory.create_session(session_id)

        # Get previous messages from last run if available
        previous_messages = []
        last_run: Run = self.memory.get_last_object(session_id)
        if last_run:
            last_state = last_run.get_final_state()
            if last_state:
                previous_messages = last_state["messages"]

        initial_state: AgentState = {
            "user_query": query,
            "instructions": self.instructions,
            "messages": previous_messages,
            "current_tool_calls": None,
            "session_id": session_id,
        }

        run_object = self.workflow.run(initial_state)

        # Store the complete run object in memory
        self.memory.add(run_object, session_id)

        return run_object

    def get_session_runs(self, session_id: Optional[str] = None) -> List[Run]:
        """Get all Run objects for a session

        Args:
            session_id: Optional session ID (uses "default" if None)

        Returns:
            List of Run objects in the session
        """
        return self.memory.get_all_objects(session_id)

    def reset_session(self, session_id: Optional[str] = None):
        """Reset memory for a specific session

        Args:
            session_id: Optional session to reset (uses "default" if None)
        """
        self.memory.reset(session_id)

tools=[retrieve_game, evaluate_retrieval, web_search]

agent = MemoryAgent(
    model_name="gpt-4o-mini",
    instructions="""
    You are UdaPlay, an AI research assistant specialized in the video game industry.
    Your role is to help users find accurate and relevant information about video games, platforms, and industry trends.
    You have access to tools for semantic search in a vector database of video game data and web search for up-to-date information. You also have access to a tool to evaluate if the retrieved documents are useful to answer the user's question. Always use the evaluation tool to evaluate the usefulness of the retrieved documents before using them to answer the user's question and giving the user the answer.
    Always aim to provide clear, concise, and well-referenced answers based on the retrieved data.
    If you cannot find the answer in the provided documents or web search results, respond with "I don't know".
    Always include citations to the sources of your information in your final answer especially when using web search results.
    When using web search results, always include proper citations in your final answer using this format: 1. Source Title - Retrieved from Source Name and include the URL if possible.
    
    Result should be in the format:
    Final Answer: <your answer here>
    Citations: <list of citations here>
    """,
    tools=tools,
    temperature=0.3,
)

def print_messages(messages: List[BaseMessage]):
    for m in messages:
        print(f" -> (role = {m.role}, content = {m.content}, tool_calls = {getattr(m, 'tool_calls', None)})")
        

In [23]:
# TODO: Invoke your agent
# - When Pokémon Gold and Silver was released?
# - Which one was the first 3D platformer Mario game?
# - Was Mortal Kombat X realeased for Playstation 5?

session_id = "AdaPlay"

questions = [
    "When Pokémon Gold and Silver was released?",
    "Which one was the first 3D platformer Mario game?",
    "Was Mortal Kombat X realeased for Playstation 5?"
]

for q in questions:
    run = agent.invoke(q, session_id)
    print(f"\nRun for question: {q} ===============================")
    messages = run.get_final_state()["messages"]
    print_messages(messages)
    print(f"\nAnswer:===============================\n")
    print(messages[-1].content)
    print("\n" + "="*50 + "\n")



[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Run for question: When Pokémon Gold and Silver was released? ===============================
 -> (role = system, content = 
    You are UdaPlay, an AI research assistant specialized in the video game industry.
    Your role is to help users find accurate and relevant information about video games, platforms, and industry trends.
    You have access to tools for semantic search in a vector database of video game data and web search for up-to-date information. You also have access to a tool to evaluate if the retrieved documents are useful to answer the user's question. Always use the evaluation tool to evaluate the usefulness of the retrieved d

In [24]:
runs = agent.get_session_runs(session_id)
for i, run_object in enumerate(runs, 1):
    print(f"\n# Question {i} - {questions[i-1]}")
    messages= run_object.get_final_state()["messages"]
    print(f"# Answer : {i} -",messages[-1].content)  # Final answer
    print("\n" + "_"*50 + "\n")


# Question 1 - When Pokémon Gold and Silver was released?
# Answer : 1 - Final Answer: Pokémon Gold and Silver was released in 1999 for the Game Boy Color.  
Citations: None needed as the information was retrieved from internal data.

__________________________________________________


# Question 2 - Which one was the first 3D platformer Mario game?
# Answer : 2 - Final Answer: The first 3D platformer Mario game is "Super Mario 64," which was released in 1996 for the Nintendo 64.  
Citations: None needed as the information was retrieved from internal data.

__________________________________________________


# Question 3 - Was Mortal Kombat X realeased for Playstation 5?
# Answer : 3 - Final Answer: Mortal Kombat X is not natively released for PlayStation 5, but it is backward compatible and can be played on the PS5. Additionally, a collection titled "Mortal Kombat: Legacy Kollection," which includes Mortal Kombat X, is set to release on PS5 on October 30, 2025.  
Citations: 1. Mort

### (Optional) Advanced

In [18]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes